# Creating a simple neural network in TensorFlow/Keras

In [ ]:
from tensorflow import keras as K
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.io import loadmat

### Function to create a Keras sequential model

In [ ]:
def getModel(nNodes, act_fn):
    
    model = K.Sequential()
    
    model.add( K.layers.Dense( nNodes[1], activation = act_fn[1], input_shape=(nNodes[0],) ) )
    
    for iLayer in range(2, len(nNodes)):
        
        model.add( K.layers.Dense( nNodes[iLayer], activation=act_fn[iLayer] ) )
        
    model.compile( optimizer = K.optimizers.Adam(0.001), loss = 'mse' )
    
    return model

### Function to get training and validation data from raw data

In [ ]:
def getShuffledData( inputs, outputs, train_fraction=0.8 ):
    
    nData = inputs.shape[0]
    
    if len(inputs.shape)==2:
        nInputs = inputs.shape[1]
    else:
        nInputs = 1
    
    if len(outputs.shape)==2:
        nOutputs = outputs.shape[1]
    else:
        nOutputs = 1
    
    indices = np.arange(nData).astype(int)
    np.random.shuffle(indices)
    
    nTrain = int(nData*train_fraction)
    nValid = nData - nTrain
    
    x_train = inputs[indices[:nTrain]].reshape((nTrain, nInputs))
    x_valid = inputs[indices[nTrain:]].reshape((nValid, nInputs))
    y_train = outputs[indices[:nTrain]].reshape((nTrain, nOutputs))
    y_valid = outputs[indices[nTrain:]].reshape((nValid, nOutputs))
    
    return x_train, y_train, x_valid, y_valid

### NN for Infectious Model

In [ ]:
data   = loadmat('Infectious_data.mat')
states = data['X']
time   = data['T']

inputs   = states[1:len(states)-1,:]
outputs  = states[2:len(states),:]

model  = getModel([7,10,7], [None,'linear','linear'])

xt, yt, xv, yv = getShuffledData(inputs, outputs)

model.fit( xt, yt, epochs=4000, batch_size=20, validation_data=(xv, yv), verbose=0)

In [ ]:
outputs_pred = model.predict( inputs )

sio.savemat('prediction.mat', mdict={'prediction': outputs_pred, 't_pred': time[1:len(states)-1,:]})

%matplotlib notebook
plt.plot(time[1:len(states)-1,:], outputs_pred, '-k')
plt.plot(time, states, '-.r')
plt.legend()
plt.show()

# Autoencoders

In [ ]:
class Autoencoder:
    
    def __init__(self, nNodes_encoder, nNodes_decoder, act_fn_encoder, act_fn_decoder):
        
        if nNodes_encoder[-1]!=nNodes_decoder[0]:
            print("Error: Code dimensions in encoder and decoder must agree")
            return
        
        if nNodes_encoder[0]!=nNodes_decoder[-1]:
            print("Error: Source/reconstruction dimensions in encoder and decoder must agree")
            return
        
        self.nNodes_encoder = nNodes_encoder
        self.nNodes_decoder = nNodes_decoder
        self.act_fn_encoder = act_fn_encoder
        self.act_fn_decoder = act_fn_decoder
        
        self.nNodes_autoencoder = []
        self.nNodes_autoencoder.extend(self.nNodes_encoder)
        self.nNodes_autoencoder.extend(self.nNodes_decoder[1:])
        
        self.act_fn_autoencoder = []
        self.act_fn_autoencoder.extend(self.act_fn_encoder)
        self.act_fn_autoencoder.extend(self.act_fn_decoder[1:])
        
        self.autoencoder = getModel(self.nNodes_autoencoder, self.act_fn_autoencoder)
        self.encoder     = getModel(self.nNodes_encoder, self.act_fn_encoder)
        self.decoder     = getModel(self.nNodes_decoder, self.act_fn_decoder)
        
    def refresh(self):
        
        iAutoencoderLayer = 0
        
        for iEncoderLayer in range(0, len(self.nNodes_encoder)-1):
            self.encoder.layers[iEncoderLayer].set_weights(self.autoencoder.layers[iAutoencoderLayer].get_weights())
            iAutoencoderLayer += 1
        
        for iDecoderLayer in range(0, len(self.nNodes_decoder)-1):
            self.decoder.layers[iDecoderLayer].set_weights(self.autoencoder.layers[iAutoencoderLayer].get_weights())
            iAutoencoderLayer += 1

### Example: Family of Gaussians

In [ ]:
x = np.linspace(0.,1.,257)
data = np.zeros((20,257))

for iData in range(20):
    data[iData] = np.exp(-(x-0.025*iData)**2 * 25)

%matplotlib notebook
plt.plot(x, data.T, '-k')
plt.show()

In [ ]:
autoencoder = Autoencoder([257, 65, 17, 3],
                          [3, 33, 129, 257],
                          [None,'sigmoid','sigmoid','linear'], 
                          [None,'sigmoid','sigmoid','linear'])

xt, yt, xv, yv = getShuffledData(data, data)

autoencoder.autoencoder.fit(xt, yt, epochs=4000, batch_size=100, validation_data=(xv, yv), verbose=0)
autoencoder.refresh()

In [ ]:
direct_reconstruction = autoencoder.autoencoder.predict(xt)

code = autoencoder.encoder.predict(xt)
indirect_reconstruction = autoencoder.decoder.predict(code)

%matplotlib notebook
plt.plot(x, direct_reconstruction.T, '-k')
plt.plot(x[::8], xt[:,::8].T, '.r')
plt.title("Reconstruction for training dataset")
plt.show()

In [ ]:
direct_reconstruction = autoencoder.autoencoder.predict(xv)

code = autoencoder.encoder.predict(xv)
indirect_reconstruction = autoencoder.decoder.predict(code)

%matplotlib notebook
plt.plot(x, direct_reconstruction.T, '-k')
plt.plot(x[::8], xv[:,::8].T, '.r')
plt.title("Reconstruction for validation dataset")
plt.show()